In [ ]:
### %matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=False)

#%% prepair data
import numpy as np
import h5py
import os
import sys
import glob

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import pandas as pd

from tqdm import tqdm

sim_path = "output/simulation/*.h5"
pkl_file = "output/analysis/data.pkl"

In [ ]:
#%% prepair data
if not os.path.isfile(pkl_file):
    os.makedirs("output/analysis", exist_ok=True)
    file_list = sorted(glob.glob(sim_path))

    # get resolution:
    with h5py.File(file_list[0], 'r') as h5f:
        for name in ['PM']:
            h5f_sub = h5f[name + '/p']
            res = h5f_sub['analysis/rofl/direction'].shape
    shape = (len(file_list), 2, res[0] * res[1])

    df = pd.DataFrame(
        columns=["f0", "f1", "omega", "psi", "rofl_dir", "rofl_inc", "rofl_trel"])

    print('collect rofl data')

    for f, file in enumerate(tqdm(file_list)):
        with h5py.File(file, 'r') as h5f:
            for name in ['PM']:
                for m, model in enumerate(['p', 'r']):
                    h5f_sub = h5f[name + '/' + model]

                    rofl_dir = h5f_sub['analysis/rofl/direction'][
                        ...].flatten()
                    rofl_inc = h5f_sub['analysis/rofl/inclination'][
                        ...].flatten()
                    rofl_trel = h5f_sub['analysis/rofl/t_rel'][
                        ...].flatten()

                    rofl_dir[rofl_dir > np.pi / 2] -= np.pi

                    psi = h5f_sub.attrs['parameter/psi']
                    omega = h5f_sub.attrs['parameter/omega']
                    f0 = h5f_sub.attrs['parameter/f0_inc']
                    f1 = h5f_sub.attrs['parameter/f1_rot']

                    df = df.append({
                        "f0": f0,
                        "f1": f1,
                        "omega": omega,
                        "psi": psi,
                        "rofl_dir":
                            rofl_dir,
                        "rofl_inc":
                            rofl_inc,
                        "rofl_trel":
                            rofl_trel
                    }, ignore_index = True)
    
    df.to_pickle(pkl_file)

In [ ]:
#%% visualize data
df = pd.read_pickle(pkl_file)

df = df.loc[df['f1'] == 0.0]
# df = df[["omega", "rofl_dir", "rofl_trel"]]
df
# g = sns.pairplot(df, hue="omega", diag_kind=None)
# g.axes[0, 0].set_xlim((-5, 95))
# g.axes[1, 0].set_xlim((-5, 95))
# g.axes[2, 0].set_xlim((-5, 95))
# plt.show()

In [ ]:
psi_list = npzfile['psi_list']
phi_list = npzfile['phi_list']
omega_list = npzfile['omega_list']

print(np.unique(psi_list))

for p in np.unique(psi_list):
    ind = np.where(np.logical_and(psi_list == p, phi_list == 0))[0]

    dp = 5
    p = p * dp

    m = 0
    plt.subplot(311)
    plt.errorbar(npzfile['omega_list'][ind] + m + p - 0.5 * dp,
                 np.rad2deg(npzfile['rofl_dir_mean'][ind, m]),
                 np.rad2deg(npzfile['rofl_dir_std'][ind, m]),
                 linestyle='None',
                 marker='^')

    plt.subplot(312)
    plt.errorbar(npzfile['omega_list'][ind] + m + p - 0.5 * dp,
                 np.rad2deg(npzfile['rofl_inc_mean'][ind, m]),
                 np.rad2deg(npzfile['rofl_inc_std'][ind, m]),
                 linestyle='None',
                 marker='^')

    plt.subplot(313)
    plt.errorbar(npzfile['omega_list'][ind] + m + p - 0.5 * dp,
                 npzfile['rofl_trel_mean'][ind, m],
                 npzfile['rofl_trel_std'][ind, m],
                 linestyle='None',
                 marker='^')
plt.show()